In [5]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

import torch

from helpers import *

In [6]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [7]:
lr = linear_model.LinearRegression()

X_train = train_input.view(train_input.size(0), -1)
X_test = test_input.view(test_input.size(0), -1)

# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
lr.fit(X=X_train, y=train_target)
preds = lr.predict(X_test)

preds = torch.FloatTensor(preds)

In [8]:
preds = discrete_predictions(preds)
accuracy = compute_accuracy(test_target, preds)
print('Accuracy ' + str(accuracy*100) + '%')

27
Accuracy 73.0%
